In [ ]:
!pip install timm torchmetrics -q
import os, torch, timm, json, cv2
import torch.nn as nn
import shutil
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
import pandas as pd
from copy import deepcopy
from sklearn.model_selection import StratifiedShuffleSplit
from pathlib import Path

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 19.1 MB/s eta 0:00:00


In [ ]:
# 원본 경로
DRIVE_DIR = "/content/drive/MyDrive/집중도 체크 프로그램/data"
ZIP_PATH  = f"{DRIVE_DIR}/processed_384.zip"

# 로컬 대상 경로
LOCAL_ZIP  = "/content/processed_384.zip"

# ZIP 복사 (진행률 표시)
!rsync -ah --info=progress2 "{ZIP_PATH}" "{LOCAL_ZIP}"

         55.58G 100%   78.65MB/s    0:11:14 (xfr#1, to-chk=0/1)


In [ ]:
import zipfile

EXTRACT_DIR = "/content/processed_384"
os.makedirs(EXTRACT_DIR, exist_ok=True)

with zipfile.ZipFile(LOCAL_ZIP, "r") as zip_ref:
    zip_ref.extractall(EXTRACT_DIR)

print("✅ 압축 해제 완료:", EXTRACT_DIR)

# 파일 구조 미리보기
!ls -R /content/data| head -n 30

✅ 압축 해제 완료: /content/data_pose0
ls: cannot access '/content/data': No such file or directory


In [ ]:
# 전처리 + meta 생성


ROOT = "/content/data_pose0"
OUT_ROOT = "/content/processed_384"
os.makedirs(OUT_ROOT, exist_ok=True)
SAVE_DIR = Path(OUT_ROOT)/"images"
SAVE_DIR.mkdir(parents=True, exist_ok=True)

CAND_SETS = ["A","B","C","D","E","F","G"]

rows = []
for s in CAND_SETS:
    json_dir = Path(ROOT)/f"pose0_TL_{s}_json"
    img_dir  = Path(ROOT)/f"pose0_TS_{s}_img"
    jsons = sorted(json_dir.glob("*.json"))
    for jp in tqdm(jsons, desc=f"Set {s}"):
        stem = jp.stem
        ip = img_dir/f"{stem}.png"
        if not ip.exists(): continue

        try:
            d = json.loads(open(jp, "r", encoding="utf-8").read())
            emotion = d.get("label_gt", {}).get("exp", None)
            box = d.get("label_gt", {}).get("box", None)
            if not emotion or not box: continue

            x, y, w, h = int(box["x"]), int(box["y"]), int(box["w"]), int(box["h"])
            img = cv2.imread(str(ip))
            if img is None: continue

            # bbox 기반 크롭 + 리사이즈
            cx, cy = x + w//2, y + h//2
            side = int(max(w, h) * 1.35)
            x1, y1 = max(cx - side//2, 0), max(cy - side//2, 0)
            x2, y2 = min(x1 + side, img.shape[1]), min(y1 + side, img.shape[0])
            crop = img[y1:y2, x1:x2]
            crop = cv2.resize(crop, (384, 384), interpolation=cv2.INTER_AREA)

            out_dir = SAVE_DIR/emotion
            out_dir.mkdir(parents=True, exist_ok=True)
            out_path = out_dir/f"{stem}.png"
            cv2.imwrite(str(out_path), crop)

            rows.append({"name": f"{stem}.png", "label": emotion})
        except Exception as e:
            continue

print(f"✅ 전처리 완료: {len(rows)}개 이미지 저장됨")

df = pd.DataFrame(rows)
uniq_labels = sorted(df["label"].unique())
label2id = {lab:i for i,lab in enumerate(uniq_labels)}
df["label_id"] = df["label"].map(label2id)

# Stratified split 8:1:1
sss1 = StratifiedShuffleSplit(n_splits=1, train_size=0.8, random_state=42)
train_idx, tmp_idx = next(sss1.split(df[["name"]], df["label_id"]))
df_tr, df_tmp = df.iloc[train_idx].copy(), df.iloc[tmp_idx].copy()
sss2 = StratifiedShuffleSplit(n_splits=1, train_size=0.5, random_state=43)
val_idx, test_idx = next(sss2.split(df_tmp[["name"]], df_tmp["label_id"]))
df_val, df_test = df_tmp.iloc[val_idx].copy(), df_tmp.iloc[test_idx].copy()

df_tr["split"]="train"; df_val["split"]="val"; df_test["split"]="test"
df_all = pd.concat([df_tr, df_val, df_test], ignore_index=True)
df_all.to_csv(f"{OUT_ROOT}/meta.csv", index=False)

print("✅ meta.csv 저장 완료:", f"{OUT_ROOT}/meta.csv")
print("라벨 목록:", uniq_labels)


Set G: 100%|██████████| 12380/12380 [08:04<00:00, 25.53it/s]


✅ 전처리 완료: 97299개 이미지 저장됨
✅ meta.csv 저장 완료: /content/processed_384/meta.csv
라벨 목록: ['anger', 'disgust', 'fear', 'happy', 'none', 'sad', 'surprise']


In [ ]:
# 원본 경로
DRIVE_DIR = "/content/drive/MyDrive/집중도 체크 프로그램/data"
ZIP_PATH  = f"{DRIVE_DIR}/checkpoint_latest.pth"

# 로컬 대상 경로
LOCAL_ZIP  = "/content/processed_384"

# ZIP 복사 (진행률 표시)
!rsync -ah --info=progress2 "{ZIP_PATH}" "{LOCAL_ZIP}"

        445.51M 100%   74.80MB/s    0:00:05 (xfr#1, to-chk=0/1)


In [ ]:
# 원본 경로
DRIVE_DIR = "/content/drive/MyDrive/집중도 체크 프로그램/data"
ZIP_PATH  = f"{DRIVE_DIR}/meta.csv"

# 로컬 대상 경로
LOCAL_ZIP  = "/content/processed_384"

# ZIP 복사 (진행률 표시)
!rsync -ah --info=progress2 "{ZIP_PATH}" "{LOCAL_ZIP}"

          4.46M 100%  114.23MB/s    0:00:00 (xfr#1, to-chk=0/1)


In [ ]:
# 원본 경로
DRIVE_DIR = "/content/drive/MyDrive/집중도 체크 프로그램/data"
ZIP_PATH  = f"{DRIVE_DIR}/best_model.pth"

# 로컬 대상 경로
LOCAL_ZIP  = "/content/processed_384"

# ZIP 복사 (진행률 표시)
!rsync -ah --info=progress2 "{ZIP_PATH}" "{LOCAL_ZIP}"

        111.37M 100%   34.53MB/s    0:00:03 (xfr#1, to-chk=0/1)


In [ ]:
import shutil, os

target = "/content/data_pose0"

if os.path.exists(target):
    shutil.rmtree(target)
    print(f"✅ 폴더 삭제 완료: {target}")
else:
    print(f"⚠️ 폴더가 존재하지 않음: {target}")

✅ 폴더 삭제 완료: /content/data_pose0


In [ ]:
# 경로 설정
DATA_ROOT = "/content/processed_384"
CSV_PATH  = f"{DATA_ROOT}/meta.csv"
CKPT_PATH = f"{DATA_ROOT}/checkpoint_latest.pth"
BEST_PATH = f"{DATA_ROOT}/best_model.pth"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Device:", device)


✅ Device: cuda


In [ ]:
class EmotionDataset(Dataset):
    def __init__(self, csv_path, root, split, transform):
        df = pd.read_csv(csv_path)
        self.df = df[df["split"] == split].reset_index(drop=True)
        self.root = root
        self.transform = transform

    def __len__(self): return len(self.df)

    def __getitem__(self, idx):
        r = self.df.iloc[idx]
        img_path = os.path.join(self.root, "images", r["label"], r["name"])
        image = Image.open(img_path).convert("RGB")
        if self.transform: image = self.transform(image)
        return image, int(r["label_id"])

train_tf = transforms.Compose([
    transforms.RandomResizedCrop(384, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.3,0.3,0.3,0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
val_tf = transforms.Compose([
    transforms.Resize((384,384)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

train_ds = EmotionDataset(CSV_PATH, DATA_ROOT, "train", train_tf)
val_ds   = EmotionDataset(CSV_PATH, DATA_ROOT, "val", val_tf)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=2)

num_classes = len(pd.read_csv(CSV_PATH)["label"].unique())
print(f"✅ Dataset Ready | train={len(train_ds)} | val={len(val_ds)} | classes={num_classes}")


✅ Dataset Ready | train=77839 | val=9730 | classes=7


In [ ]:
model = timm.create_model("convnext_tiny", pretrained=True, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss(label_smoothing=0.05)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20, eta_min=1e-6)

# EMA 복사
ema = deepcopy(model)
for p in ema.parameters():
    p.requires_grad_(False)

def ema_update(src, dst, decay=0.999):
    with torch.no_grad():
        for p, q in zip(dst.parameters(), src.parameters()):
            p.data.mul_(decay).add_(q.data, alpha=1-decay)

# ✅ 체크포인트 로드 함수
def save_ckpt(epoch, best_acc):
    torch.save({
        "epoch": epoch,
        "model": model.state_dict(),
        "optimizer": optimizer.state_dict(),
        "scheduler": scheduler.state_dict(),
        "ema": ema.state_dict(),
        "best_acc": best_acc
    }, CKPT_PATH)

def load_ckpt():
    ckpt = torch.load(CKPT_PATH, map_location=device)
    model.load_state_dict(ckpt["model"])
    optimizer.load_state_dict(ckpt["optimizer"])
    scheduler.load_state_dict(ckpt["scheduler"])
    ema.load_state_dict(ckpt["ema"])
    return ckpt["epoch"] + 1, ckpt["best_acc"]

start_epoch, best_acc = 1, 0.0
if os.path.exists(CKPT_PATH):
    start_epoch, best_acc = load_ckpt()
    print(f"🔁 체크포인트 불러옴 | epoch={start_epoch}, best_acc={best_acc:.4f}")
else:
    print("🆕 새 학습 세션 시작")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

🔁 체크포인트 불러옴 | epoch=25, best_acc=0.8963


In [ ]:
EPOCHS = 40
patience, wait = 6, 0

for epoch in range(start_epoch, EPOCHS + 1):
    model.train()
    tr_loss, tr_correct, tr_total = 0.0, 0, 0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS} [Train]")
    for x, y in pbar:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        ema_update(model, ema)

        tr_loss += loss.item() * y.size(0)
        tr_correct += (out.argmax(1) == y).sum().item()
        tr_total += y.size(0)
        pbar.set_postfix(loss=f"{tr_loss/tr_total:.4f}", acc=f"{tr_correct/tr_total:.3f}")
    scheduler.step()

    # ✅ Validation
    ema.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0
    with torch.no_grad():
        for x, y in tqdm(val_loader, desc=f"Epoch {epoch}/{EPOCHS} [Val]"):
            x, y = x.to(device), y.to(device)
            out = ema(x)
            loss = criterion(out, y)
            val_loss += loss.item() * y.size(0)
            val_correct += (out.argmax(1) == y).sum().item()
            val_total += y.size(0)
    val_acc = val_correct / val_total

    print(f"📊 Epoch {epoch} 완료 | ValAcc={val_acc:.4f}")
    save_ckpt(epoch, best_acc)

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(ema.state_dict(), BEST_PATH)
        print(f"🌟 Best 모델 갱신: {best_acc:.4f}")
        wait = 0
    else:
        wait += 1
        if wait >= patience:
            print("⏹ Early Stopping!")
            break


Epoch 25/40 [Val]: 100%|██████████| 305/305 [00:40<00:00,  7.58it/s]


📊 Epoch 25 완료 | ValAcc=0.8957


Epoch 26/40 [Val]: 100%|██████████| 305/305 [00:34<00:00,  8.78it/s]


📊 Epoch 26 완료 | ValAcc=0.8955


Epoch 27/40 [Val]: 100%|██████████| 305/305 [00:36<00:00,  8.32it/s]


...

📊 Epoch 28 완료 | ValAcc=0.8928


Epoch 29/40 [Val]: 100%|██████████| 305/305 [00:35<00:00,  8.69it/s]


📊 Epoch 29 완료 | ValAcc=0.8957


Epoch 30/40 [Val]: 100%|██████████| 305/305 [00:34<00:00,  8.78it/s]


📊 Epoch 30 완료 | ValAcc=0.8935
⏹ Early Stopping!


## 테스트셋 평가

In [ ]:
test_tf = transforms.Compose([
    transforms.Resize(384),
    transforms.CenterCrop(384),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225]),
])

csv_path = "/content/processed_384/meta.csv"          # 네가 쓰는 csv 경로
root = "/content/processed_384"                       # 이미지 루트
batch_size = 128

test_ds = EmotionDataset(csv_path, root, split="test", transform=test_tf)
test_loader = DataLoader(test_ds, batch_size=batch_size,
                         shuffle=False, num_workers=4, pin_memory=True)

print("테스트셋 개수:", len(test_ds))

테스트셋 개수: 9730


In [ ]:
NUM_CLASSES = 7  # 👉 감정 클래스 개수에 맞게 숫자만 바꿔줘

def create_convnext_tiny(num_classes: int):
    # 학습할 때도 거의 이렇게 썼을 가능성이 큼
    model = timm.create_model(
        'convnext_tiny',
        pretrained=False,        # 이미 학습 끝난 weight를 로드할 거라 False
        num_classes=num_classes
    )
    return model


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_classes = 7  # 감정 클래스 수에 맞게
model = create_convnext_tiny(num_classes)  # 네가 사용한 모델 생성 함수/클래스

best_ckpt_path = "/content/processed_384/best_model.pth"  # 실제 경로로 수정

state = torch.load(best_ckpt_path, map_location=device)
model.load_state_dict(state)
model.to(device)

print("✅ best 모델 로드 완료")

✅ best 모델 로드 완료


In [ ]:
import numpy as np        # ← 이거 추가
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
def evaluate_on_test(model, loader, device, class_names=None):
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            out = model(x)
            preds = out.argmax(1)

            all_preds.append(preds.cpu().numpy())
            all_targets.append(y.cpu().numpy())

    all_preds = np.concatenate(all_preds)
    all_targets = np.concatenate(all_targets)

    # 전체 정확도
    acc = (all_preds == all_targets).mean()
    print(f"\n🎯 Test Accuracy: {acc:.4f}")

    # Confusion Matrix
    cm = confusion_matrix(all_targets, all_preds)
    print("\n🧩 Confusion Matrix:")
    print(cm)

    # 클래스 이름 있으면 리포트 예쁘게
    if class_names is not None:
        print("\n📊 Classification Report:")
        print(classification_report(all_targets, all_preds, target_names=class_names))
    else:
        print("\n📊 Classification Report (label index 기준):")
        print(classification_report(all_targets, all_preds))

    return acc, cm, all_targets, all_preds

# 감정 라벨 이름 (네 데이터 기준으로 수정)
class_names = ["angry", "disgust", "fear", "happy", "sad", "surprise", "neutral"]

test_acc, cm, y_true, y_pred = evaluate_on_test(model, test_loader, device, class_names)


🎯 Test Accuracy: 0.8979

🧩 Confusion Matrix:
[[1148   80   62    8   15   12   42]
 [  66 1224   53    2    5   58    0]
 [  48   69 1055    4   17   62   18]
 [   4    0    2 1507    2    6   19]
 [   8    5   15    2 1419   11   40]
 [   7   43   72    2    8 1103    3]
 [  30    1   24   19   42    7 1281]]

📊 Classification Report:
              precision    recall  f1-score   support

       angry       0.88      0.84      0.86      1367
     disgust       0.86      0.87      0.87      1408
        fear       0.82      0.83      0.83      1273
       happy       0.98      0.98      0.98      1540
         sad       0.94      0.95      0.94      1500
    surprise       0.88      0.89      0.88      1238
     neutral       0.91      0.91      0.91      1404

    accuracy                           0.90      9730
   macro avg       0.89      0.90      0.89      9730
weighted avg       0.90      0.90      0.90      9730

